In [1]:
import cv2
import numpy as np

class VisualOdometry:
    def __init__(self, focal_length, principal_point):
        """
        Инициализация визуальной одометрии.
        :param focal_length: Фокусное расстояние камеры.
        :param principal_point: Главная точка камеры (оптический центр).
        """
        self.focal = focal_length
        self.pp = principal_point
        self.prev_image = None
        self.prev_keypoints = None
        self.R = np.eye(3)  # Матрица вращения
        self.t = np.zeros((3, 1))  # Вектор трансляции

    def process_frame(self, curr_image):
        if self.prev_image is None:
            self.prev_image = curr_image
            self.prev_keypoints = self.detect_keypoints(curr_image)
            return self.t.flatten()

        # Найти соответствия между предыдущими и текущими ключевыми точками.
        curr_keypoints, matches = self.track_keypoints(self.prev_image, curr_image, self.prev_keypoints)

        if len(curr_keypoints) < 5 or len(self.prev_keypoints) < 5:
            self.prev_image = curr_image
            self.prev_keypoints = self.detect_keypoints(curr_image)
            # print("Недостаточно ключевых точек для вычисления движения.")
            return self.t.flatten()

        # Вычислить относительное движение между изображениями.
        try:
            E, mask = cv2.findEssentialMat(curr_keypoints, self.prev_keypoints, focal=self.focal, pp=self.pp, method=cv2.RANSAC)
            _, R, t, mask = cv2.recoverPose(E, curr_keypoints, self.prev_keypoints, focal=self.focal, pp=self.pp)
        except Exception as e:
            self.prev_image = curr_image
            self.prev_keypoints = self.detect_keypoints(curr_image)
            return self.t.flatten()



        # Обновить глобальную матрицу вращения и трансляции.
        self.t += self.R @ t
        self.R = R @ self.R

        self.prev_image = curr_image
        self.prev_keypoints = curr_keypoints[mask.ravel() == 1]
        self.prev_keypoints = self.detect_keypoints(curr_image)

        return self.t.flatten()


    def detect_keypoints(self, image):
        """
        Обнаружение ключевых точек на изображении.
        :param image: Входное изображение.
        :return: Массив ключевых точек.
        """
        orb = cv2.ORB_create()
        keypoints = orb.detect(image, None)
        keypoints = np.array([kp.pt for kp in keypoints], dtype=np.float32)
        return keypoints

    def track_keypoints(self, prev_image, curr_image, prev_keypoints):
        # Оптическое отслеживание ключевых точек
        try:
            curr_keypoints, status, err = cv2.calcOpticalFlowPyrLK(prev_image, curr_image, prev_keypoints, None)
        except Exception as e:
            return [], []
        # print(curr_keypoints)
        # print(status)
        # print(err)

        # Проверка на случай, если status равен None
        if status is None or curr_keypoints is None:
            print("Не удалось отследить ключевые точки.")
            return [], []

        # Фильтрация успешных ключевых точек
        status = status.ravel()
        prev_keypoints = prev_keypoints[status == 1]
        curr_keypoints = curr_keypoints[status == 1]

        return curr_keypoints, prev_keypoints



cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Параметры камеры (замените на свои значения)
focal_length = 718.856  # Примерное значение фокусного расстояния
principal_point = (320, 240)  # Предположим, что камера 640x480
vo = VisualOdometry(focal_length=focal_length, principal_point=principal_point)

size = 400
mult = 5

traj = np.zeros(shape=(size, size, 3), dtype=np.uint8)
traj_2 = np.zeros(shape=(size, size, 3), dtype=np.uint8)
traj_3 = np.zeros(shape=(size, size, 3), dtype=np.uint8)

x_prev = 0
y_prev = 0
z_prev = 0
threshold = 0.5

calc_x = 0
calc_y = 0
calc_z = 0

while True:
    ret, frame = cap.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Отображение текущего кадра
    # cv2.imshow('frame', frame)

    # Преобразование изображения в градации серого
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # try:
        # Обработка текущего кадра
    position = vo.process_frame(img)
    x = position[0]
    y = position[1]
    z = position[2]

    if abs(x - x_prev) >= threshold:
        calc_x += x - x_prev
    if  abs(y - y_prev) >= threshold:
        calc_y += y - y_prev
    if  abs(z - z_prev) >= threshold:
        calc_z += z - z_prev

    x_prev = x
    y_prev = y
    z_prev = z

    # print(f"Текущая позиция камеры: x={x:.2f}, y={y:.2f}, z={z:.2f}")
    # Обновляем траектории
    traj = cv2.circle(traj, (int(calc_x * mult) + size//2, int(calc_z * mult) + size//2), 1, (0, 255, 0), 1)
    traj_2 = cv2.circle(traj_2, (int(calc_x * mult) + size//2, int(calc_y * mult) + size//2), 1, (0, 255, 0), 1)
    traj_3 = cv2.circle(traj_3, (int(calc_y * mult) + size//2, int(calc_z * mult) + size//2), 1, (0, 255, 0), 1)

    # Добавляем текст на траекторию
    cv2.putText(traj, 'Trajectory X-Z', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(traj_2, 'Trajectory X-Y', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(traj_3, 'Trajectory Y-Z', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Изменяем размер текущего кадра до размера траекторий
    resized_frame = cv2.resize(frame, (size, size))

    # Объединяем изображения в одну матрицу
    top_row = np.hstack((resized_frame, traj))
    bottom_row = np.hstack((traj_2, traj_3))
    combined_image = np.vstack((top_row, bottom_row))

    # Отображаем объединенное изображение
    cv2.imshow('Visual Odometry', combined_image)

        # Нажмите 'q', чтобы выйти из цикла
    if cv2.waitKey(1) == ord('q'):
        break

    if cv2.waitKey(1) == ord('c'):
        traj = np.zeros(shape=(size, size, 3))
        traj_2 = np.zeros(shape=(size, size, 3))
        traj_3 = np.zeros(shape=(size, size, 3))
        vo.t = np.zeros((3, 1))

cap.release()
cv2.destroyAllWindows()